<img style="float: center;" src="lacaja.jpg">

In [1]:
from collections import defaultdict
import re
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd   
from gensim import corpora, models, similarities, matutils
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.matutils import cossim
from gensim.models.phrases import Phrases, Phraser
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import nltk

import pickle
import random
import sys
from tqdm import tqdm
from sklearn import manifold
import time
import gc

start_time = time.time()

%matplotlib inline

from playsound import playsound

C:\Users\paladugu\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import random
random.seed(100)
random_state = 100

In [3]:
# cambio el display para observar bien el contenido de las rows
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', -1)

C:\Users\paladugu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [4]:
path_df = "C:/Users/paladugu/Downloads/nps-generalli/nps-generalli/data/original/"
path_processed = "C:/Users/paladugu/Downloads/nps-generalli/nps-generalli/data/processed/"
figures = "C:/Users/paladugu/Downloads/nps-generalli/nps-generalli/figures/"
sonido = "C:/Users/paladugu/Downloads/nps-generalli/nps-generalli/sound/"
sys.path.append("C:/Users/paladugu/Downloads/nps-generalli/nps-generalli/source")

In [5]:
df_2020 = pd.read_csv(path_df +"nps_enero2019_febrero_2021.csv",error_bad_lines=False,  encoding='ISO-8859-1', sep=';')
print(f"Tamaño del Dataset: {df_2020.shape}")

C:\Users\paladugu\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Tamaño del Dataset: (125605, 176)


In [6]:
# Filtro columnas con un umbral determinado
limitPer = len(df_2020) *0.50 #Cantidad de filas del dataset * el porcentahe umbral
df = df_2020.dropna(thresh=limitPer,axis=1) #dropna() con la opción del umbral y que filtre rows
df.head().T


,0,1,2,3,4
CREATION_DATE,5/12/2018 10:30:04,5/12/2018 10:30:04,6/12/2018 10:30:04,6/12/2018 10:30:04,10/12/2018 10:30:53
RESPONSE_DATE,2/1/2019 15:46:19,3/1/2019 12:54:57,4/1/2019 12:35:15,1/1/2019 18:48:25,7/1/2019 02:20:20
CURRENT_STATUS,Resolved,NaN,Closed,NaN,Closed
ALL_LOG_NOTES_COMBINED,[2019-01-02 10:46:48] Alert Created: Detractor Consumer: AR; [2019-01-02 10:46:50] Alert status set to New; [2019-01-03 03:32:08] Case Edited: Dueño de este caso (por favor ingresar su nombre) · Fecha del Primer contacto · Hora del Primer contacto · Resultado del Primer contacto · Agenda de próximo contacto (araujo); [2019-01-03 03:59:07] Case Edited: Date of 2nd Feedback Call · Time of 2nd Feedback Call · Outcome of 2nd Feedback Call (araujo); [2019-01-03 04:17:49] Case Edited: Date of 3rd Feedback Call · Time of 3rd Feedback Call · Outcome of 3rd Feedback Call · Descripción detallada del contacto (araujo); [2019-01-03 08:05:51] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-01-03 08:05:57] Alert Closed (araujo); [2019-01-03 08:06:47] Note Added: 3 intentos de contacto fallidos. se envía mail para una próxima llamada. 5TA LLAMADA SIN EXITO. SE RE ENVIA MAIL DONDE SE LE SOLICITA PEDIR BAJA DEL SEGURO DE AP para dar curso al trámite (araujo); [2019-05-21 04:18:02] Alert Reopened (araujo); [2019-05-21 04:18:06] Alert Resolved (araujo); [2019-05-21 04:18:23] Alert Closed (araujo); [2019-05-21 06:43:03] Alert Reopened (araujo); [2019-05-21 06:43:36] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-05-21 06:43:42] Alert Resolved (araujo); [2019-05-21 06:43:54] Alert Closed (araujo); [2019-05-21 06:44:00] Alert Reopened (araujo); [2019-05-21 06:44:04] Alert Resolved (araujo),NaN,[2019-01-04 07:35:23] Alert Created: Promoter Autoclose: AR; [2019-01-04 07:35:25] Alert Closed,NaN,[2019-01-06 21:20:29] Alert Created: Detractor Consumer: AR; [2019-01-06 21:20:31] Alert status set to New; [2019-01-07 09:01:25] Case Edited: Dueño de este caso (por favor ingresar su nombre) · Fecha del Primer contacto · Hora del Primer contacto · Resultado del Primer contacto · Agenda de próximo contacto · Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución · Acciones derivadas a otros departamentos · Categorías de llamadas con comentarios negativos · Categorías de llamadas con comentarios positivos (ferraro); [2019-01-07 09:01:30] Alert Closed (ferraro)
ALERT_STATUS_CHANGED_TIME,21/5/2019 06:44:04,NaN,4/1/2019 07:35:25,NaN,7/1/2019 09:01:30
TITLE,Estimado,Estimado,Estimado,Estimado,Estimado
DOCUMENT_TYPE,DNI,DNI,DNI,DNI,DNI
DOCUMENT_NUMBER,24257783,13947489,12017332,13018239,21852196
FIRST_NAME,GUSTAVO JORGE ARIEL,LUCAS ARMANDO,JUAN JOSE,DANIEL ANGEL,ALEJANDRO MARCELO
LAST_NAME,SAMANIEGO,DIAZ,ESCOBAR,MENJIBAR,BUTERA


## Normalización

In [7]:
df.CASE_DATE = pd.to_datetime(df.CASE_DATE,errors = 'coerce', format = '%d/%m/%Y')
df["mes"] = df.CASE_DATE.dt.month

C:\Users\paladugu\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\paladugu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
df["año"] = df.CASE_DATE.dt.year

C:\Users\paladugu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
del df_2020
del limitPer

In [10]:
df.categoria = ""

In [11]:
df.TNPS_SCORE.unique()

array([ 0,  7, 10,  8,  5,  2,  9,  1,  6,  4,  3], dtype=int64)

In [12]:
lst = [df]
for column in lst:
    column.loc[column["TNPS_SCORE"] <= 6,  'categoria'] = "detractor"
    column.loc[(column["TNPS_SCORE"] >= 7) & (column["TNPS_SCORE"] <= 8), 'categoria'] = "neutro"
    column.loc[(column["TNPS_SCORE"] >= 9), 'categoria'] = "promotor"


In [13]:
df.head().T

,0,1,2,3,4
CREATION_DATE,5/12/2018 10:30:04,5/12/2018 10:30:04,6/12/2018 10:30:04,6/12/2018 10:30:04,10/12/2018 10:30:53
RESPONSE_DATE,2/1/2019 15:46:19,3/1/2019 12:54:57,4/1/2019 12:35:15,1/1/2019 18:48:25,7/1/2019 02:20:20
CURRENT_STATUS,Resolved,NaN,Closed,NaN,Closed
ALL_LOG_NOTES_COMBINED,[2019-01-02 10:46:48] Alert Created: Detractor Consumer: AR; [2019-01-02 10:46:50] Alert status set to New; [2019-01-03 03:32:08] Case Edited: Dueño de este caso (por favor ingresar su nombre) · Fecha del Primer contacto · Hora del Primer contacto · Resultado del Primer contacto · Agenda de próximo contacto (araujo); [2019-01-03 03:59:07] Case Edited: Date of 2nd Feedback Call · Time of 2nd Feedback Call · Outcome of 2nd Feedback Call (araujo); [2019-01-03 04:17:49] Case Edited: Date of 3rd Feedback Call · Time of 3rd Feedback Call · Outcome of 3rd Feedback Call · Descripción detallada del contacto (araujo); [2019-01-03 08:05:51] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-01-03 08:05:57] Alert Closed (araujo); [2019-01-03 08:06:47] Note Added: 3 intentos de contacto fallidos. se envía mail para una próxima llamada. 5TA LLAMADA SIN EXITO. SE RE ENVIA MAIL DONDE SE LE SOLICITA PEDIR BAJA DEL SEGURO DE AP para dar curso al trámite (araujo); [2019-05-21 04:18:02] Alert Reopened (araujo); [2019-05-21 04:18:06] Alert Resolved (araujo); [2019-05-21 04:18:23] Alert Closed (araujo); [2019-05-21 06:43:03] Alert Reopened (araujo); [2019-05-21 06:43:36] Case Edited: Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución (araujo); [2019-05-21 06:43:42] Alert Resolved (araujo); [2019-05-21 06:43:54] Alert Closed (araujo); [2019-05-21 06:44:00] Alert Reopened (araujo); [2019-05-21 06:44:04] Alert Resolved (araujo),NaN,[2019-01-04 07:35:23] Alert Created: Promoter Autoclose: AR; [2019-01-04 07:35:25] Alert Closed,NaN,[2019-01-06 21:20:29] Alert Created: Detractor Consumer: AR; [2019-01-06 21:20:31] Alert status set to New; [2019-01-07 09:01:25] Case Edited: Dueño de este caso (por favor ingresar su nombre) · Fecha del Primer contacto · Hora del Primer contacto · Resultado del Primer contacto · Agenda de próximo contacto · Descripción detallada del contacto · Fuiste capaz de resolver el inconveniente? · El cliente se encontró satisfecho con el resultado? · Hay riesgos de que el cliente solicite la baja de su póliza? · El cliente desea ser contactado por un representante de Generali? · El cliente recibirá un regalo sorpresa? · Plan de acción para la resolución · Acciones derivadas a otros departamentos · Categorías de llamadas con comentarios negativos · Categorías de llamadas con comentarios positivos (ferraro); [2019-01-07 09:01:30] Alert Closed (ferraro)
ALERT_STATUS_CHANGED_TIME,21/5/2019 06:44:04,NaN,4/1/2019 07:35:25,NaN,7/1/2019 09:01:30
TITLE,Estimado,Estimado,Estimado,Estimado,Estimado
DOCUMENT_TYPE,DNI,DNI,DNI,DNI,DNI
DOCUMENT_NUMBER,24257783,13947489,12017332,13018239,21852196
FIRST_NAME,GUSTAVO JORGE ARIEL,LUCAS ARMANDO,JUAN JOSE,DANIEL ANGEL,ALEJANDRO MARCELO
LAST_NAME,SAMANIEGO,DIAZ,ESCOBAR,MENJIBAR,BUTERA


In [14]:
df.CASE_DATE = pd.to_datetime(df.CASE_DATE,errors = 'coerce', format = '%d/%m/%Y')

In [15]:
df["año"] = df.CASE_DATE.dt.year

In [16]:
import re
import string

In [17]:
df.TNPS_COMMENT.head()

0    Pedí seguro de mí auto y me agregaron otros seguros que no pedí.                                                                                                                                                                                                                                                                                                                                                                                          
1    7                                                                                                                                                                                                                                                                                                                                                                                                                                                         
2    CALIDAD DEL SERVICIO                                                                               

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125605 entries, 0 to 125604
Data columns (total 44 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   CREATION_DATE              125605 non-null  object        
 1   RESPONSE_DATE              125605 non-null  object        
 2   CURRENT_STATUS             98771 non-null   object        
 3   ALL_LOG_NOTES_COMBINED     108857 non-null  object        
 4   ALERT_STATUS_CHANGED_TIME  98771 non-null   object        
 5   TITLE                      125605 non-null  object        
 6   DOCUMENT_TYPE              125605 non-null  object        
 7   DOCUMENT_NUMBER            125605 non-null  int64         
 8   FIRST_NAME                 125124 non-null  object        
 9   LAST_NAME                  125602 non-null  object        
 10  GENDER                     125035 non-null  object        
 11  DAY_OF_BIRTH               124994 non-null  object  

In [19]:
df.TNPS_COMMENT = df.TNPS_COMMENT.astype(str) 	

In [20]:
def basic_text_cleanup(text):
    """
    Convert to lowercase, remove punctuation, remove digits
    """
    text = text.lower() #lower case
    text = re.sub("[^\w|\s]", " ",text) #replace non word and non space characters by a space
    text = re.sub("\s+", " ",text) #remove extra spaces
    text = re.sub("á", "a",text) #saco acentos
    text = re.sub("é", "e",text) #saco acentos
    text = re.sub("í", "i",text) #saco acentos
    text = re.sub("ó", "o",text) #saco acentos
    text = re.sub("ú", "u",text) #saco acentos
    text = re.sub(",", "",text) #saco comas
    #doc = nlp(text)#convert to a spacy object
    #doc = [w.text for w in doc if not w.is_stop or w.text =="no"] # remove stopwords but keep "no" for bigrams
    #doc = [w for w in doc if (len(w)> 2 or w == "no")] #only words longer than 2 characters
    #doc = [w for w in doc if (len(w)> 2 )] #only words longer than 2 characters
    #doc = [w for w in doc if w != "y"] #remove "y"
    #doc = " ".join(doc)
    return text

In [21]:
df["clean_text"] = df["TNPS_COMMENT"].apply(lambda x:basic_text_cleanup(x) )

In [22]:
 df["TNPS_COMMENT"].tail()

125600    Rapidez en la solicitud de mi pedido                                                    
125601    Rapida solucion al problema                                                             
125602    por la buena atencion que tienen , y la buena resolucion de los problemas               
125603    Se ha dado cumplimiento en casi todo lo que se prometio con la contratacion del servicio
125604    Muy buena atencion                                                                      
Name: TNPS_COMMENT, dtype: object

In [24]:
df["clean_text"].tail()

125600    rapidez en la solicitud de mi pedido                                                    
125601    rapida solucion al problema                                                             
125602    por la buena atencion que tienen y la buena resolucion de los problemas                 
125603    se ha dado cumplimiento en casi todo lo que se prometio con la contratacion del servicio
125604    muy buena atencion                                                                      
Name: clean_text, dtype: object

### Normalización 2.0
Quiero agarrar todos los sinonimos de bueno, buena, bueno, excelente, hacer un listado y reemplazar todo por Bueno y todo lo que sea malo, malisimo, mierda, cagada, por malo. 

I want to take all the synonyms for good, good, good, excellent, make a list and replace everything with Good and everything that is bad, bad, shit, shit, bad.

In [25]:
def normalizo(text):
    text = re.sub(r"\brapid[a-zA-Z]*|celeridad","rapidez",text)
    text = re.sub(r"\befic[a-zA-Z]*","eficiencia",text)
    text =re.sub(r"\baten[a-zA-Z]*","atencion",text)
    text =re.sub(r"\bauxil[a-zA-Z]*","auxilio",text)
    text =re.sub(r"\bbuen[a-zA-Z]*","buen",text)
    text =re.sub(r"\bproblem[a-zA-Z]*","problema",text)
    #text =re.sub(r"\bmal[a-zA-Z]*","malo",text)
    text =re.sub(r"\bcumpli[a-zA-Z]*","cumplimiento",text)
    text = re.sub(r"auxilio mecanico|grua","auxilio", text)
    text = re.sub(r"respondieron|responder|respuestas|responden","respuesta", text)
    text = re.sub(r"buena|excelencia|exelente|bien|buen|excelente|copado|excelente|apropiado|lujo|eselente|exelente|buenoisimo","bueno", text) #
    text = re.sub(r"mal[a-zA-Z]*|desastre|desastrozo|mierda|falta de compromiso","malo", text)
    
    return text

In [26]:
df["normal"] = df["clean_text"].apply(lambda x:normalizo(x) )


In [30]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [35]:
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize

In [36]:
#nltk.download()
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paladugu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [37]:
STOP_WORDS = stopwords.words('spanish')

def tokenize(x):
    return [word for word in word_tokenize(x) if word not in STOP_WORDS and word.isalpha() and len(word)>1]
    

texts= [tokenize(doc.lower()) for doc in df["normal"]]


In [38]:
df["token"] = df["normal"].apply(lambda x:tokenize(x))

In [39]:
df["token"].head()

0    [pedi, seguro, auto, agregaron, seguros, pedi]                                                                                                                                                                                                                                                                                                     
1    []                                                                                                                                                                                                                                                                                                                                                 
2    [calidad, servicio]                                                                                                                                                                                                                                                                                              

In [40]:
df.to_csv(path_processed+"comments.csv")

In [42]:
import gc
print("--- %s segundos---" % (time.time() - start_time))
playsound(sonido+"war.mp3")
gc.collect()


--- 1482.4722120761871 segundos---


29